In [4]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [7]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'home appliances' # topic of the search
dataset = '&sp=3814' # dataset: industry news
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [8]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following (please write in total minimum 155 words): 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem Generation:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem Generation:

- "Title": Energy-Efficient Smart Appliances
- "Explanation": The problem lies in the lack of widespread adoption of energy-efficient smart appliances in households, despite the availability of advanced technologies that can optimize energy consumption and improve overall efficiency. This gap represents a missed opportunity for both consumers seeking to reduce energy costs and environmental impact, as well as manufacturers looking to meet evolving market demands for sustainable solutions.
- "User group": Environmentally conscious consumers seeking cost-effective solutions for managing household energy consumption.
- "Pain point": These users face the challenge of balancing sustainability goals with practical considerations of cost and convenience, leading to a lack of accessible options that effectively combine energy efficiency with smart features in home appliances.

---

Problem Generation:

- "Title": Energy-Efficient Smart Appliances
- "Explanation": With the 

In [9]:
# second agent:

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria. Please be very critical in your ratings and only give a 3 rating when it is very very very good:
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 20 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

Title: Energy-Efficient Smart Appliances

Size of user group: 3 (large target group)
Relevancy to home appliances industry: 3 (very relevant to users)
Specificity of the problem: 3 (specific and not generic problem)

Justification: The problem affects a large group of environmentally conscious consumers seeking efficient solutions, directly impacting the home appliances industry with a specific challenge.

Overall score: 9/9

---

Problem rating:

Problem: Energy-Efficient Smart Appliances

Rating:
- Size of user group: 3 (large target group)
- Relevancy of pain point to the industry: 3 (very relevant to users)
- Specificity of the problem: 3 (specific and not generic problem)

Justification: Energy efficiency is a pressing concern for homeowners and environmentally conscious consumers in the home appliances industry.

Overall Score: 9/9

---

Problem rating:

Title: Smart Home Appliances Security Vulnerabilities

- Size of user group: 3 (large target group) - Many hom

In [10]:
# third agent 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
        - The problem (please write 100-150 words)
        - Which user group is impacted by this problem the most (please write max 25 words)
        - What is the specific pain point for this user group resulting from the problem (please write 40-60 words)
        - One sentence that summarises the three selection criteria (Size of user group / Relevancy of pain point to the specific industry of {query} and the respective user group / How specific is the problem)

    """
    
completion = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print("Best Problem:\n")
print(completion.choices[0].message.content)
print("\n---\n")




Best Problem:

The problem with the highest overall rating is "Energy-Efficient Smart Appliances."

- The problem: The lack of widespread adoption of energy-efficient smart appliances in households, despite available advanced technologies, represents a missed opportunity for both consumers and manufacturers.
- User group impacted: Environmentally conscious consumers seeking cost-effective solutions for managing household energy consumption.
- Specific pain point: These users face challenges balancing sustainability goals with cost and convenience, leading to a lack of accessible options that effectively combine energy efficiency with smart features in home appliances.
- Summary of selection criteria: This problem affects a large target group of environmentally conscious consumers, is highly relevant to the home appliances industry, and addresses a specific challenge in combining energy efficiency with smart features.

---

